In [8]:
pip install tensorflow

     -------------------------------------- 272.8/272.8 MB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 KB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 7.5 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 7.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     -------------------------------------- 440.7/440.7 KB 6.9 MB/s eta 0:00:00
     -------------------------------------- 420.6/420.6 KB 6.6 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 KB 7.3 MB/

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\kiefe\\AppData\\Local\\Programs\\Python\\Python39\\Lib\\site-packages\\~umpy\\.libs\\libopenblas64__v0.3.21-gcc_10_3_0.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\users\kiefe\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [7]:
# importing some usefull librariies

import glob
from tqdm.auto import tqdm
import random
import pandas as pd
import numpy as np
import os
from itertools import chain
import glob
import shutil
import tensorflow as tf

from PIL import Image
import os.path
import imagehash
import multiprocessing


pd.set_option('display.max_colwidth', None)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
DATA_DIR = 'data'

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(DATA_DIR)  # Import dataset from data_dir into a image dataset

In [ ]:
hashing_method = [imagehash.average_hash,imagehash.phash,imagehash.whash,imagehash.dhash] # Here defining a list of hashes we will use

def return_hash(self,row_idx): # here we assume that the column name which contains frame  name is titled 'frame_name'
    '''
    INPUT
    row_idx: int; row number of dataframe containing frame name
    
    OUTPUT
    hashes of image: strings
    '''
    Hash = [] # creating an empty list to store the hash of each image based upon all the hashing method defined above
    img_read = Image.open(os.path.join(img_real_dir,row_idx['frame_name']))
    for func_hash in hashing_method:
        Hash.append(str(func_hash(img_read))) 

    return Hash[0],Hash[1],Hash[2],Hash[3] # returning the hash of an image based upon different algorithms

# we have defined a function which will return the hashes of image and 




tqdm.pandas() # we will use multi processing to engage all cores of cpu
df_dum = df.copy() # keeping a copy of dataframe
rows_iter = (row for _, row in df_dum.iterrows()) # row list 
pool = multiprocessing.Pool() # initiating the multiprocessing pool

# Here we map the function defined above to all the rows(frames) inside dataframe and create four columns titles hash1, hash2, hash3, hash4
# corresponding to the hash algoithms defined above
df_dum['hash1'],df_dum['hash2'],df_dum['hash3'],df_dum['hash4'] = zip(*list(tqdm(pool.imap(return_hash,rows_iter),total=len(df_dum)))) 

pool.close()
pool.join()

# now we will create a set of hash1 column and remove the duplicate rows and then further apply it to remaning hashes

df_dum = df_dum.groupby(['hash1'], as_index=False)[['frame_name','hash2','hash3','hash4']].agg(lambda x: list(set(x)))

df_dum['hash2'] = df_dum['hash2'].apply(lambda x: x[0]) # this returns the remaining hashes in a list containg duplicate hashes hence we are 
# taking the first hash out of list for hash2

df_dum = df_dum.groupby(['hash2'], as_index=False)[['frame_name','hash3','hash4']].agg(lambda x: list(set(list(chain.from_iterable(x)))))
df_dum['hash3'] = df_dum['hash3'].apply(lambda x: x[0])

df_dum = df_dum.groupby(['hash3'], as_index=False)[['frame_name','hash4']].agg(lambda x: list(set(list(chain.from_iterable(x)))))
df_dum['hash4'] = df_dum['hash4'].apply(lambda x: x[0])

df_dum = df_dum.groupby(['hash4'], as_index=False)[['frame_name']].agg(lambda x: list(set(list(chain.from_iterable(x)))))

df_dum['frame_name'] = df_dum['frame_name'].apply(lambda x: x[0])
df_dum.drop(columns='hash4') # removing the last column


In [ ]:
df_dum # it is the final dataframe containg unique imgaes name